In [108]:
'''

@Author: Vighnesh Harish Bilgi
@Date: 2022-12-12
@Last Modified by: Vighnesh Harish Bilgi
@Last Modified time: 2022-12-12
@Title : 3 - Connect S3 datasets to redshift using AWS Glue Job

'''

'\n\n@Author: Vighnesh Harish Bilgi\n@Date: 2022-12-12\n@Last Modified by: Vighnesh Harish Bilgi\n@Last Modified time: 2022-12-12\n@Title : 3 - Connect S3 datasets to redshift\n\n'

In [119]:
import boto3
import pandas as pd
import json
import time
from io import StringIO
import redshift_connector

In [120]:
import os
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'
os.environ['AWS_ACCESS_KEY_ID'] = os.environ.get('test1_access_key')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.environ.get('test1_secret_access_key')

In [121]:
COVID_BUCKET = "covid19-schema"
redshift_cluster_password = os.environ.get('redshift_cluster_password')

### Connect to redshift_connector

In [122]:
conn = redshift_connector.connect(
    host = 'redshift-cluster-1.c0gcoo9i6xgc.us-east-1.redshift.amazonaws.com',
    database='dev',
    user = 'awsuser',
    password="Chinmay_123"
)

### Set auto-commit to True

In [123]:
conn.autocommit = True

### Fetching redshift cursor

In [124]:
cursor = redshift_connector.Cursor = conn.cursor()

### Query to create table 'dim_date'

In [132]:
cursor.execute(""" CREATE TABLE IF NOT EXISTS "dim_date" (
  "fips" REAL,
  "date" TIMESTAMP,
  "year" INTEGER,
  "month" INTEGER,
  "day_of_week" INTEGER
) """)

### COPY command to get records from dimDate.csv from S3 bucket

In [133]:
cursor.execute(""" COPY dev.public.dim_date FROM 's3://covid19-schema/output/dimDate.csv' 
IAM_ROLE 'arn:aws:iam::949401335332:role/MyRedshiftRole' 
FORMAT AS CSV DELIMITER ',' 
QUOTE '"' 
IGNOREHEADER 1 
REGION AS 'us-east-1' """)


### Query to create table 'dim_hospital'

In [134]:
cursor.execute(""" CREATE TABLE IF NOT EXISTS "dim_hospital" (
"fips" INTEGER,
  "state_name" TEXT,
  "latitude" REAL,
  "longtitude" REAL,
  "hq_address" TEXT,
  "hospital_name" TEXT,
  "hq_city" TEXT,
  "hq_state" TEXT
) """)

### COPY command to get records from dimHospital.csv from S3 bucket

In [135]:
cursor.execute(""" COPY dev.public.dim_hospital FROM 's3://covid19-schema/output/dimHospital.csv' 
IAM_ROLE 'arn:aws:iam::949401335332:role/MyRedshiftRole' 
FORMAT AS CSV DELIMITER ',' 
QUOTE '"' 
IGNOREHEADER 1 
REGION AS 'us-east-1' """)

### Query to create table 'dim_region'

In [125]:
cursor.execute(""" CREATE TABLE IF NOT EXISTS "dim_region" (
"fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "latitude" REAL,
  "longitude" REAL,
  "county" TEXT,
  "state" TEXT
) """)

### COPY command to get records from dimRegion.csv from S3 bucket

In [126]:
cursor.execute(""" COPY dev.public.dim_region FROM 's3://covid19-schema/output/dimRegion.csv' 
IAM_ROLE 'arn:aws:iam::949401335332:role/MyRedshiftRole' 
FORMAT AS CSV DELIMITER ',' 
QUOTE '"' 
IGNOREHEADER 1 
REGION AS 'us-east-1' """)

### Query to create table 'fact_covid'

In [130]:
cursor.execute(""" CREATE TABLE IF NOT EXISTS "fact_covid" (
"fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "confirmed" REAL,
  "deaths" REAL,
  "recovered" REAL,
  "active" REAL,
  "date" TIMESTAMP,
  "positive" INTEGER,
  "negative" REAL,
  "hospitalized" REAL,
  "hospitalizedcurrently" REAL,
  "hospitalizeddischarged" REAL,
  "year" INTEGER,
  "month" INTEGER,
  "day_of_week" INTEGER
) """)

### COPY command to get records from factCovid.csv from S3 bucket

In [131]:
cursor.execute(""" COPY dev.public.fact_covid FROM 's3://covid19-schema/output/factCovid.csv' 
IAM_ROLE 'arn:aws:iam::949401335332:role/MyRedshiftRole' 
FORMAT AS CSV DELIMITER ',' 
QUOTE '"' 
IGNOREHEADER 1 
REGION AS 'us-east-1' """)